In [1]:
%%time
#    This file is part of Multi-objective optimization under uncertainty research

import array, copy, random
# import logging
# import random
import numpy as np

# imports for the BNN
import torch
import torch.nn as nn
import torch.nn.functional as F

import keras
from keras import backend as K

import pickle

from deap import algorithms, base, creator, tools


# load min and max values of the data to denormalize prediction data
with open('maxmin.pickle', 'rb') as f:
    [max_x, min_x, max_y, min_y] = pickle.load(f)
    
# load min and max values of the data to denormalize prediction data
with open('maxmin_thickness.pickle', 'rb') as f:
    [X_max, X_min, Y_max, Y_min] = pickle.load(f)    

def normalize_max_min(data, data_max, data_min):
    return (data-data_min) / (data_max-data_min)

def denormalize_max_min(data, data_max, data_min):
    return data * (data_max-data_min) + data_min

class KerasDropoutPrediction(object):
    def __init__(self,model):
        self.f = K.function(
                [model.layers[0].input, 
                 K.learning_phase()],
                [model.layers[-1].output])
        
    def predict(self, x, n_iter=10):
        result = []
        for _ in range(n_iter):
            result.append(self.f([x, 1]))
        result = np.array(result).reshape(n_iter,len(x)).T
        return result
    
class MC_Dropout_Model(nn.Module):
    def __init__(self, input_dim, output_dim, num_units, drop_prob):
        super(MC_Dropout_Model, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.drop_prob = drop_prob

        # network with two hidden and one output layer
        self.layer1 = nn.Linear(input_dim, num_units)
        self.layer2 = nn.Linear(num_units, num_units)
        self.layer3 = nn.Linear(num_units, 2 * output_dim)

        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = x.view(-1, self.input_dim)

        x = self.layer1(x)
        x = self.activation(x)
        x = F.dropout(x, p=self.drop_prob, training=True)

        x = self.layer2(x)
        x = self.activation(x)
        x = F.dropout(x, p=self.drop_prob, training=True)

        x = self.layer3(x)

        return x

    
    
# load BL model BNN and evaluate objectives
model_BL = torch.load('BNN_BLmodel.pt')

# load the thickness model
model_thickness = keras.models.load_model('MCdropout_model_thickness.h5', compile=False)

# predict with dropout
kdp = KerasDropoutPrediction(model_thickness)



def evaluate(vars):

    # Minimize(abs(pred_mean – target))
    target  = 4.2 # desired part thickness in mm

    # number of total layers = (maximum part height)/(height of a layer), i.e., 4.2 / (layer height)
    if vars[2] == 1:
        height = 0.42
    elif vars[2] == 2:
        height = 0.6
    elif vars[2] == 3:
        height = 0.7

        
        
    #  MODEL BOND LENGTH  
    # print(vars)
    num_layers = np.int(target / height); # number of layers

    num_interfaces = 14     # number of interfaces per layer
    width = 0.8             # filament width in mm

    ycoord = 0.5 * height  # 0.5*height of a layer in mm
    iki_y = ycoord * 2
    
    # Create an input array to predict overall part thickness
    inp_BL = [] # input to BNN to make predictions
    
    # store inputs for GP(model disrepancy at each interface)
    for jj in range(1, num_layers + 1):
        for ii in range(1, num_interfaces + 1):
            # use x & y coordinates of vertical bonds as training data for the GP
            # Inp =[ Temperature, speed, height, x, y ]
            inp_BL.append([vars[0], vars[1], height, ii * width, ycoord + (jj - 1) * iki_y])

    # Convert built Python lists to a Numpy array.
    inp_BL = np.array(inp_BL, dtype='float32')

    # normalize data
    inp_BL = normalize_max_min(inp_BL, max_x, min_x)

    x_pred = torch.tensor(inp_BL)  # convert to torch tensor

    samples = []
    noises = []
    for i in range(30):
        preds = model_BL.forward(x_pred).cpu().data.numpy()
        samples.append(denormalize_max_min(preds[:, 0], max_y, min_y))
        noises.append(denormalize_max_min(np.exp(preds[:, 1]), max_y, min_y))

    samples, noises = np.array(samples),  np.array(noises)
    means = (samples.mean(axis=0)).reshape(-1)

    aleatoric = (noises ** 2).mean(axis=0) ** 0.5
    epistemic = (samples.var(axis=0) ** 0.5).reshape(-1)
    total_unc = (aleatoric ** 2 + epistemic ** 2) ** 0.5


    # Dimensionless BL: non-dimensionalize the BL by dividing with the layer height
    dimensionless_mean_bl = means.mean()/height
    dimensionless_total_unc_bl = total_unc.mean()/height**2



    # MODEL THICKNESS
    x_pos = 7 # mm
    num_iter = (10.5-1.5)/0.01 + 1
        
     # Create an input array to predict overall part thickness
    inp_thickness = []
    
    # store inputs for GP(model disrepancy at each interface)
    for jj in range(5):
        y_pos = 1.5 # mm
        for ii in range(int(num_iter)):
            # use x & y coordinates of vertical bonds as training data for the GP
            # Inp =[ Temperature, speed, height, x, y]
            inp_thickness.append([vars[0], vars[1], height, x_pos, y_pos])
            y_pos += 0.01 # increment y position 0.01 mm
        x_pos += 5 # x coordinate  
    
    # Convert built Python lists to a Numpy array.
    inp_thickness = np.array(inp_thickness, dtype='float32')

    # normalize data
    inp_thickness = normalize_max_min(inp_thickness, X_max, X_min)
    
    # Predict
    y_pred_do = kdp.predict(inp_thickness,n_iter=50)
    y_pred_do_org = denormalize_max_min(y_pred_do, Y_max, Y_min)
    y_pred_do_org_mean = y_pred_do_org.mean(axis=1).reshape(-1, 1)
    y_pred_do_org_std = y_pred_do_org.std(axis=1).reshape(-1, 1)
    
    # Predicted mean and std part thicknesses
    mean_part_thickness = y_pred_do_org_mean.mean()
    std_part_thickness = ((y_pred_do_org_std**2).mean())**0.5
    
    
#     return 1-dimensionless_mean_bl, dimensionless_total_unc_bl # model bond length
#     return abs(mean_part_thickness-target), std_part_thickness # model thickness
    return 1-dimensionless_mean_bl, abs(mean_part_thickness-target) # model BL and thickness


# The constraint is:
# (Nozzle velocity) x (line width) x (layer thickness)  less than/ equal to 24 mm/s3
def feasible(individual):
    """Feasability function for the individual. Returns True if feasible False
    otherwise."""
    
    line_width = 0.8 # in mm
    
    # layer height in mm
    if individual[2] == 1:
        height = 0.42
    elif individual[2] == 2:
        height = 0.6
    elif individual[2] == 3:
        height = 0.7
        
    if individual[1] * line_width * height <= 24:
#         print(individual,'true')
        return True
#     print(individual,'false')
    return False

IND_SIZE = 3
N_CYCLES = 1
BOUND_LOW, BOUND_UP = [217, 26, 1], [278, 44, 3]

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin, n=IND_SIZE)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_temperature", random.randint, 217, 278)
toolbox.register("attr_speed", random.randint, 26, 44)
toolbox.register("attr_layer", random.randint, 1, 3)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_temperature,toolbox.attr_speed,toolbox.attr_layer), n=N_CYCLES)

# Structure initializers
# toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
# # load BL model BNN and evaluate objectives
# toolbox.model1 = torch.load('BNN_BLmodel.pt')
# toolbox.register("evaluate", evaluate)

# # load the thickness model
# model_thickness = keras.models.load_model('MCdropout_model_thickness.h5', compile=False)

# # predict with dropout
# toolbox.model2 = KerasDropoutPrediction(model_thickness)

# A penality function can be added to any evaluation function using the DeltaPenality decorator provided in the tools module.
# Delta = [0,1] worst cases if maximize obj1 and min. obj2
# toolbox.decorate("evaluate", tools.DeltaPenality(feasible, [0,1]))
# Delta = [1,1] worst cases if min. obj1 and min. obj2
toolbox.decorate("evaluate", tools.DeltaPenality(feasible, [1,1]))

toolbox.register("mate", tools.cxUniform, indpb=0.50)
toolbox.register("mutate", tools.mutUniformInt, low=BOUND_LOW, up=BOUND_UP, indpb=0.50)

def checkBounds(min, max):
    def decorator(func):
        def wrappper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max[i]:
#                         print(child[i])
                        child[i] = max[i]
                    elif child[i] < min[i]:
#                         print(child[i])
                        child[i] = min[i]
            return offspring
        return wrappper
    return decorator

# Bounds on the design variables
toolbox.decorate("mate", checkBounds([217, 26, 1], [278, 44, 3]))
toolbox.decorate("mutate", checkBounds([217, 26, 1], [278, 44, 3]))

toolbox.register("select", tools.selNSGA2)
# ref_points = tools.uniform_reference_points(nobj=2, p=12)
# toolbox.register("select", tools.selNSGA3WithMemory(ref_points))

toolbox.pop_size = 100
toolbox.max_gen = 20
toolbox.mut_prob = 0.3

stats = tools.Statistics()
stats.register("pop", copy.deepcopy)

def run_ea(toolbox, stats=None, verbose=True):
    pop = toolbox.population(n=toolbox.pop_size)
    pop = toolbox.select(pop, len(pop))
    return algorithms.eaMuPlusLambda(pop, toolbox, mu=toolbox.pop_size, 
                                     lambda_=toolbox.pop_size, 
                                     cxpb=1-toolbox.mut_prob,
                                     mutpb=toolbox.mut_prob, 
                                     stats=stats, 
                                     ngen=toolbox.max_gen, 
                                     verbose=verbose)


Using TensorFlow backend.
C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

Instructions for updating:
Colocations handled automatically by placer.
Wall time: 2.74 s


C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\torch\serialization.py:420: UserWarning: Couldn't retrieve source code for container of type MC_Dropout_Model. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


## Run the multi-objective evolutionary algorithm

In [2]:
%time pop, logbook = run_ea(toolbox, stats=stats)

gen	nevals	pop                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

2  	100   	(Individual('d', [278.0, 29.0, 1.0]), Individual('d', [269.0, 26.0, 2.0]), Individual('d', [278.0, 44.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [271.0, 33.0, 2.0]), Individual('d', [267.0, 38.0, 1.0]), Individual('d', [257.0, 37.0, 1.0]), Individual('d', [276.0, 44.0, 1.0]), Individual('d', [267.0, 31.0, 2.0]), Individual('d', [266.0, 40.0, 1.0]), Individual('d', [266.0, 29.0, 2.0]), Individual('d', [261.0, 37.0, 1.0]), Individual('d', [252.0, 26.0, 2.0]), Individual('d', [270.0, 29.0, 1.0]), Individual('d', [266.0, 28.0, 2.0]), Individual('d', [266.0, 28.0, 2.0]), Individual('d', [223.0, 27.0, 1.0]), Individual('d', [227.0, 32.0, 1.0]), Individual('d', [226.0, 31.0, 1.0]), Individual('d', [237.0, 34.0, 1.0]), Individual('d', [267.0, 41.0, 1.0]), Individual('d', [258.0, 38.0, 1.0]), Individual('d', [260.0, 39.0, 1.0]), Individual('d', [266.0, 28.0, 2.0]), Individual('d', [266.0, 28.0, 2.0]), Individual('d', [260.0, 40.0, 2.0]), Individual('d', [263.0, 40

5  	100   	(Individual('d', [269.0, 26.0, 2.0]), Individual('d', [218.0, 27.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [269.0, 29.0, 2.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [278.0, 35.0, 2.0]), Individual('d', [269.0, 29.0, 2.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [273.0, 26.0, 2.0]), Individual('d', [273.0, 26.0, 2.0]), Individual('d', [218.0, 27.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [278.0, 44.0, 1.0]), Individual('d', [266.0, 40.0, 1.0]), Individual('d', [276.0, 44.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [257.0, 37.0, 1.0]), Individual('d', [257.0, 37.0, 1.0]), Individual('d', [271.0, 33.0, 2.0]), Individual('d', [271.0, 33.0, 2.0]), Individual('d', [261.0, 37.0, 1.0]), Individual('d', [263.0, 38.0, 1.0]), Individual('d', [261.0, 37.0, 1.0]), Individual('d', [278.0, 29.0, 1.0]), Individual('d', [274.0, 29

8  	100   	(Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 27.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [218.0, 27.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [269.0, 33.0, 1.0]), Individual('d', [269.0, 33.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [269.0, 26

11 	100   	(Individual('d', [266.0, 32.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [218.0, 28

14 	100   	(Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [217.0, 28

17 	100   	(Individual('d', [270.0, 33.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [217.0, 28

20 	100   	(Individual('d', [270.0, 33.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [266.0, 32.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [217.0, 26.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [218.0, 28.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [270.0, 33.0, 1.0]), Individual('d', [271.0, 33.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [217.0, 28.0, 1.0]), Individual('d', [266.0, 32

## Plot the Pareto Front

In [3]:
### import file from another folder ###
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Python-Save-Plots')
import save_plots as sP
# import SaveFigAsPDF_PGF as sF

In [4]:
front = np.array([ind.fitness.values for ind in pop])
# plt.scatter(front[:,0], front[:,1], c="b")
# plt.axis("tight")
# plt.xlabel('$f_1()$')
# plt.ylabel('$f_2()$')
# plt.show()

# Save plots
# legends = ['Train', 'Test']


# filename = 'pareto_front_BL'
# labels = ['$1-\mu_{\mathrm{\hat{bl}}}$', '$\sigma_{\mathrm{\hat{bl}}}$']

# filename = 'pareto_front_thickness'
# labels = ['$\sigma_{\mathrm{th}}$', '$\sigma_{\mathrm{th}}$']

filename = 'pareto_front_BLvsThickness'
labels = ['$1-\mu_{\mathrm{\hat{bl}}}$', '$\mu_{\mathrm{th}}$']
    
x1, y1 = front[:,0], front[:,1]

# row: number of lines, col.: dimension of the plot (e.g., (1,2) -> 1 line with x and y values)
num_lines = 1
dim_plot = 2

# initialize array with size number of lines
data = np.full((num_lines,dim_plot), None)
data[0] = [x1,y1]

sP.run_subplot(data, labels, filename, plot_type=2)
    

C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\torch\lib;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\bin;;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\mingw-w64\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\usr\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Scripts;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\bin;C:\Users\berkc\Miniconda3\condabin;C:\Program Files\Git\cmd;C:\Program Files\MATLAB\R2019b\bin;C:\Users\berkc\Miniconda3;C:\Users\berkc\Miniconda3\Scripts;C:\Users\berkc\Miniconda3\envs\Research_AM_2020;C:\Users\berkc\Miniconda3\envs\Research_AM_2020\Scripts;C:\WINDOWS\system32;C:\Users\berkc\AppData\Local\Programs\Python\Python37-32\Scripts;C:\Users\berkc\AppData\Local\Programs\Python\Python37-32;C:\WINDOWS\system32;C:\Users\berkc\AppData\Local\Programs\MiKTeX

## Hypervolume computation

Sort the best individuals in each local Pareto optimal sets and add a vector (2D; each dimension for each objectives respectively) to obtain the reference point (nadir), worst case.

In [5]:
fronts = [tools.sortLogNondominated(pop, k=len(pop), first_front_only=True) 
          for pop in logbook.select('pop')]

# Reference point
# max. obj1 & min. obj2
# reference = np.max([np.max([ind.fitness.values for ind in front], axis=0) for front in fronts], axis=0) + [-0.3, 0.1]
# min. obj1 & min. obj2
reference = np.max([np.max([ind.fitness.values for ind in front], axis=0) for front in fronts], axis=0) + [0.3, 0.1]
print(reference)

[0.46702956 0.28734043]


In [6]:
import deap.benchmarks.tools as bt

Compute and plot hypervolume indicator wrt. number of generations

In [7]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# plt.rc('text', usetex=True)

hypervols = [bt.hypervolume(front, reference) for front in fronts]

# plt.figure(figsize=(7, 4))
# plt.plot(hypervols)
# plt.xlabel('Generations')
# plt.ylabel('Hypervolume');
# plt.show()


# Save plots
# legends = ['Train', 'Test']
# filename = 'Hypervolume_vs_generations_bl'
# filename = 'Hypervolume_vs_generations_th'
filename = 'Hypervolume_vs_generations_bl_th'
labels = ['Generations', 'Hypervolume']
# x1, y1 = range(1,len(hypervols)+1), hypervols
x1 = hypervols

# row: number of lines, col.: dimension of the plot (e.g., (1,2) -> 1 line with x and y values)
num_lines = 1
dim_plot = 1

# initialize array with size number of lines
data = np.full((num_lines,dim_plot), None)

data[0] = [x1]

sP.run_subplot(data, labels, filename, plot_type=1)

C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\lib\site-packages\torch\lib;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\bin;;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\mingw-w64\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\usr\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Library\bin;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\Scripts;C:\Users\berkc\Miniconda3\envs\Clone_Research_AM_2020\bin;C:\Users\berkc\Miniconda3\condabin;C:\Program Files\Git\cmd;C:\Program Files\MATLAB\R2019b\bin;C:\Users\berkc\Miniconda3;C:\Users\berkc\Miniconda3\Scripts;C:\Users\berkc\Miniconda3\envs\Research_AM_2020;C:\Users\berkc\Miniconda3\envs\Research_AM_2020\Scripts;C:\WINDOWS\system32;C:\Users\berkc\AppData\Local\Programs\Python\Python37-32\Scripts;C:\Users\berkc\AppData\Local\Programs\Python\Python37-32;C:\WINDOWS\system32;C:\Users\berkc\AppData\Local\Programs\MiKTeX

Sort the pareto front individuals wrt. obj1

In [8]:
p_fronts = np.array([[ind,ind.fitness.values] for ind in pop])
p_fronts[p_fronts[:,1].argsort()] # sort by column BL

array([[Individual('d', [217.0, 26.0, 1.0]),
        (0.1087426032338823, 0.043056458597347635)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10886465083985097, 0.04309480656750431)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10893745365596952, 0.043090967684057624)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10897456464313326, 0.043042329655796685)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10905127014432636, 0.04311851708490266)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10905907551447547, 0.043112161128484594)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.1090779503186543, 0.04309850025687201)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10915366240910118, 0.04310568624654376)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10916778303328012, 0.043079293683500275)],
       [Individual('d', [217.0, 26.0, 1.0]),
        (0.10918594825835448, 0.043062543569023504)],
       [Individu

# Animation

In [9]:
# Animation

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
#     plt.rcParams['text.latex.preamble'] = '\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')
import pandas as pd

from matplotlib import animation
from IPython.display import HTML

def animate(frame_index, logbook):
    'Updates all plots to match frame _i_ of the animation.'
    print("Frame Index: ", frame_index)
    ax.clear()
    plot_colors = seaborn.color_palette("Set1", n_colors=10)
    
    # get the Pareto fronts in the population (pop).
    fronts = tools.emo.sortLogNondominated(logbook.select('pop')[frame_index],
                                           len(logbook.select('pop')[frame_index]));
    for i, inds in enumerate(fronts):
        par = [toolbox.evaluate(ind) for ind in inds]
        df = pd.DataFrame(par)
        df.plot(ax=ax, kind='scatter', label='Front ' + str(i + 1),
                x=df.columns[0], y=df.columns[1], alpha=0.47,
                color=plot_colors[i % len(plot_colors)])

    ax.set_title('$t=$' + str(frame_index))
#     ax.set_xlabel('$1-\mu_{\mathrm{\hat{bl}}}$')
#     ax.set_ylabel('$\sigma_{\mathrm{\hat{bl}}}$')
#     ax.set_xlabel('$\mu_{\mathrm{th}}$')
#     ax.set_ylabel('$\sigma_{\mathrm{th}}$')
    ax.set_xlabel('$1-\mu_{\mathrm{\hat{bl}}}$')
    ax.set_ylabel('$\mu_{\mathrm{th}}$')

    return []

fig = plt.figure(figsize=(6,4))
ax = fig.gca()

anim = animation.FuncAnimation(fig, lambda i: animate(i, logbook),
                                   frames=len(logbook), interval=20,
                                   blit=True)
plt.close()
HTML(anim.to_html5_video())

Frame Index:  0
Frame Index:  0
Frame Index:  0
Frame Index:  1
Frame Index:  2
Frame Index:  3
Frame Index:  4
Frame Index:  5
Frame Index:  6
Frame Index:  7
Frame Index:  8
Frame Index:  9
Frame Index:  10
Frame Index:  11
Frame Index:  12
Frame Index:  13
Frame Index:  14
Frame Index:  15
Frame Index:  16
Frame Index:  17
Frame Index:  18
Frame Index:  19
Frame Index:  20
